Импортирование библиотек

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.models as models
import matplotlib.pyplot as plt

Подготовка данных

In [ ]:
# Подготовка данных
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = CIFAR10(root='./data', train=True, download=False, transform=transform)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)

test_set = CIFAR10(root='./data', train=False, download=False, transform=transform)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)
